This notebook shows different visualizations of the algorithms performances

In [ ]:
import config

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from fold import readFold
import matplotlib.pyplot as plt
from pmlb import classification_dataset_names

# Open all the score

If all methods are not computed at the same date, indicates several dates in the list

In [ ]:
dates = ['18 March 2019 15:55:08', '19 March 2019 15:33:18']

In [ ]:
def fromDictToFrame(dictionary, keys = []):
    """
        Transfroms a multidimensional dictionary in a dataframe
    """
    df = {}
    for key, value in dictionary.items():
        if isinstance(value, dict):
            df = {**df, **fromDictToFrame(value, keys + [key])}
        else:
            df[tuple(keys + [key])] = [value]
    return pd.DataFrame.from_dict(df)

def openFiles(dates):
    """
        Opens all files at the given dates and restructures it in several dataframes for clarity
    """
    scores, assignation, constraints, info = [], [], [], {}
    for date in dates:
        print("Opening {}".format(date))
        path = os.path.join(config.result, date)
        scoresDate, assignationDate, constraintsDate = {}, {}, {}
        
        # Open each dataset
        for dname in os.listdir(path):
            unpick = pickle.load(open(os.path.join(path, dname), 'rb'))

            # Open Score
            scoreDname = fromDictToFrame(unpick["Score"]).reset_index()
            if not scoreDname.empty:
                scoresDate[dname] = scoreDname

                # Open Assignation
                assignationDate[dname] = fromDictToFrame(unpick["Assignation"])

                # Open Constraints
                constraintsDate[dname] = fromDictToFrame(unpick["Percentage Constraint"])

                # Open info
                info[dname] = fromDictToFrame(readFold(unpick["Name"]))

        scores.append(pd.concat(scoresDate, axis = "index").reset_index(level=[1], drop=True))
        assignation.append(pd.concat(assignationDate, axis = "index").reset_index(level=[1], drop=True))
        constraints.append(pd.concat(constraintsDate, axis = "index").reset_index(level=[1], drop=True))

    return pd.concat(scores, join = "inner", axis = 1),\
        pd.concat(assignation, join = "inner", axis = 1),\
        pd.concat(constraints, join = "inner", axis = 1),\
        pd.concat(info, axis = "index").reset_index(level=[1], drop=True)

In [ ]:
scores, assignation, constraints, info = openFiles(dates)
print("Open {} result files".format(len(scores)))

In [ ]:
scores.head()

In [ ]:
assignation.head()

In [ ]:
constraints.head()

In [ ]:
info.head()

# Comparison

In [ ]:
metric = "Adjusted Rand"
category = "test" # 'all', 'train' or 'test'

## Global performance

Computes mean of the different iterations

In [ ]:
scores = scores.groupby(axis=1, level=[1, 2, 3]).mean()

In [ ]:
scores

Compute the global performances of the different methods

In [ ]:
measure = scores.loc[:, (slice(None), category, metric)]
measure = measure.T.reset_index(level=[1, 2], drop=True).T
sns.boxplot(data = measure, orient="h")
plt.ylabel("Methods")
plt.xlabel(metric)

In [ ]:
sns.violinplot(data = measure, orient="h")
plt.ylabel("Methods")
plt.xlabel(metric)

In [ ]:
info["Number points"] = info['Labels'].applymap(lambda x: len(x)) 
info["Number classes"] = info["N_Classes"]
cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)

grid = sns.PairGrid(data = measure)
grid = grid.map_upper(sns.scatterplot, sizes=(10, 200), size=info["Number points"], hue=info["Number classes"], palette = cmap, alpha = 0.7)
grid = grid.map_diag(sns.distplot)
grid = grid.map_lower(sns.kdeplot)

## First ranked

Compute and display the rank of the different methods

In [ ]:
var = [v for v in measure.columns if v != "Bayesian Optimization"]

In [ ]:
# Count rank position
rank = measure[var].rank(axis = 1, method = 'min',ascending = False).apply(lambda x: x.value_counts()).T 
rank = rank.sort_values(by=[1, 2, 3], ascending = False) # Reorder
rank.plot.barh(stacked=True)
plt.xlabel("Number dataset")
plt.ylabel("Methods")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Rank")

# Analysis by method

In [ ]:
method = "lcvqe"
metric = "Adjusted Rand"

## Train vs Test Performances

Compare performances between training and test, it is interesting to notice how our method does not overfit where other methods perform well on training but have bad performances on test.

In [ ]:
measure = scores.loc[:, (method, slice(None), metric)]
measure = measure.T.reset_index(level=[0, 2], drop=True).T

In [ ]:
plt.title(method)
plt.grid(alpha = 0.1)
sns.scatterplot(x="train", y="test", sizes=(10, 200), data=measure, 
                size=info["Number points"], hue=info["Number classes"], palette = cmap, alpha = 0.7)
plt.xlim(-0.1,1.1)
plt.ylim(-0.1,1.1)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

## Train vs Test Constraint

Compare the percentage of cosntraint respected on train and test

In [ ]:
from constraint import verification_constraint

In [ ]:
def percentageConstraint(gtLabels, label, trainIndices, test = False):
    """
        Compute the percentage of constraint respected
        gtLabels -- Ground truth label
        label -- Ground truth label
        trainIndices -- Indices for train
        test -- Compute on test supset if True
    """
    if test:
        trainIndices = [i for i in range(len(gtLabels)) if i not in trainIndices]
    gtConstraint = 2*np.equal.outer(gtLabels[trainIndices], gtLabels[trainIndices]) - 1
    
    verified, notVerified = verification_constraint(gtConstraint, label[trainIndices])
    return  verified / (notVerified + verified)

In [ ]:
measure["% train constraint respected"] = pd.DataFrame.from_dict({dname: np.mean([percentageConstraint(info.loc[dname]['Labels', pd.NaT],
                                                                    assignation.loc[dname][i, method],
                                                                    info.loc[dname]['Train', i, method])
                                                                for i in assignation.loc[dname].index.levels[0]])
                                                        for dname in measure.index}, orient = 'index')

measure["% test constraint respected"] = pd.DataFrame.from_dict({dname: np.mean([percentageConstraint(info.loc[dname]['Labels', pd.NaT],
                                                                    assignation.loc[dname][i, method],
                                                                    info.loc[dname]['Train', i, method], test = True)
                                                                for i in assignation.loc[dname].index.levels[0]])
                                                        for dname in measure.index}, orient = 'index')

In [ ]:
plt.title(method)
plt.grid(alpha = 0.1)
sns.scatterplot(x="% train constraint respected", y="% test constraint respected", sizes=(10, 200), data=measure, 
                size=info["Number points"], hue=info["Number classes"], palette = cmap, alpha = 0.7)
plt.xlim(-0.1,1.1)
plt.ylim(-0.1,1.1)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
plt.title(method)
plt.grid(alpha = 0.1)
sns.scatterplot(x="% train constraint respected", y="test", sizes=(10, 200), data=measure, 
                size=info["Number points"], hue=info["Number classes"], palette = cmap, alpha = 0.7)
plt.xlim(-0.1,1.1)
plt.ylim(-0.1,1.1)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

-----

# Evolution

In [ ]:
dates = ['18 March 2019 16:10:35']

In [ ]:
scores, assignation, constraints, info = openFiles(dates)
print("Open {} result files".format(len(scores)))

## Performances

In [ ]:
metric = "Adjusted Rand"
category = "test" # 'all', 'train' or 'test'

In [ ]:
measure = scores.loc[:, (slice(None), slice(None), category, metric)].stack(level = 0)
measure['Percentage Constraint'] = constraints.stack().max(axis = 1)
measure = measure.T.reset_index(level=[1, 2], drop=True).T
measure = measure.reset_index(level=[1], drop=True)

In [ ]:
measure.head()

In [ ]:
for i in info[('Name', pd.NaT)].unique():
    currentDataset = measure[info[('Name', pd.NaT)] == i]
    currentDataset["Percentage Constraint"] = currentDataset["Percentage Constraint"].apply(lambda x: round(x, 2))
    currentDataset = currentDataset.groupby("Percentage Constraint").mean().sort_index()
    currentDataset.plot()
    plt.title(i)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))